# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load data from Part I to create dataframe
weather_map = pd.read_csv('../WeatherPy/output_data/current_weather_data.csv')
weather_map.dropna(inplace = True) 
weather_map.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yellowknife,62.46,-114.35,-34.60,73,90,5.82,CA,1608209715
1,cape town,-33.93,18.42,77.00,50,20,21.92,ZA,1608209461
2,butaritari,3.07,172.79,81.95,80,100,15.17,KI,1608209718
3,kavieng,-2.57,150.80,83.71,73,100,7.81,PG,1608209719
4,khatanga,71.97,102.50,-27.83,82,0,7.96,RU,1608209720


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_map[["Lat", "Lng"]]

# Set the weight value
humidity = weather_map["Humidity"]

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Create Figure
fig = gmaps.figure( layout=figure_layout, center=(46.0, -5.0), zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities to fit weather conditions.
ideal_weather_data = weather_map.loc[(weather_map["Max Temp"] > 70) & (weather_map["Max Temp"] < 80) & (weather_map["Wind Speed"] < 10) & (weather_map["Cloudiness"] == 0)]
ideal_weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,riyadh,24.69,46.72,78.80,22,0,8.05,SA,1608209646
87,sur,22.57,59.53,72.50,56,0,7.07,OM,1608209821
92,flinders,-34.58,150.86,75.99,1,0,3.00,AU,1608209827
328,jiwani,25.05,61.74,70.79,29,0,5.61,PK,1608210121
365,vallenar,-28.57,-70.76,73.81,45,0,5.59,CL,1608210166
443,acajutla,13.59,-89.83,77.65,70,0,5.12,SV,1608209972
485,iralaya,15.00,-83.23,78.35,85,0,7.63,HN,1608210314
492,nelson bay,-32.72,152.15,75.20,88,0,4.70,AU,1608210322


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df.
hotel_df = ideal_weather_data

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
35,riyadh,24.69,46.72,78.80,22,0,8.05,SA,1608209646,
87,sur,22.57,59.53,72.50,56,0,7.07,OM,1608209821,
92,flinders,-34.58,150.86,75.99,1,0,3.00,AU,1608209827,
328,jiwani,25.05,61.74,70.79,29,0,5.61,PK,1608210121,
365,vallenar,-28.57,-70.76,73.81,45,0,5.59,CL,1608210166,
443,acajutla,13.59,-89.83,77.65,70,0,5.12,SV,1608209972,
485,iralaya,15.00,-83.23,78.35,85,0,7.63,HN,1608210314,
492,nelson bay,-32.72,152.15,75.20,88,0,4.70,AU,1608210322,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng to iterate through the weather
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging with keyword hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing information...Next Location")  

Missing information...Next Location
Missing information...Next Location


In [8]:
#View Hotel Dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
35,riyadh,24.69,46.72,78.80,22,0,8.05,SA,1608209646,Four Seasons Hotel Riyadh At Kingdom Center
87,sur,22.57,59.53,72.50,56,0,7.07,OM,1608209821,Sur Grand Hotel
92,flinders,-34.58,150.86,75.99,1,0,3.00,AU,1608209827,
328,jiwani,25.05,61.74,70.79,29,0,5.61,PK,1608210121,Hamza hotel jiwani
365,vallenar,-28.57,-70.76,73.81,45,0,5.59,CL,1608210166,Hotel Cecil
443,acajutla,13.59,-89.83,77.65,70,0,5.12,SV,1608209972,Villavela Hotel Boutique
485,iralaya,15.00,-83.23,78.35,85,0,7.63,HN,1608210314,
492,nelson bay,-32.72,152.15,75.20,88,0,4.70,AU,1608210322,Oaks Pacific Blue Resort


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer
fig.add_layer(markers)

# Display Map
fig



Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…